In [ ]:
!pip install tensorflow

In [ ]:
from sklearn.model_selection import train_test_split
from google.colab import drive
drive.mount('C://Users/Mirza/Downloads')


UNZIP THE DATASET

In [ ]:
# Unzip the dataset
!unzip "C://Users/Mirza/Downloads/flowers.zip"

# Set the path to the dataset
dataset_dir = "C://Users/Mirza/Downloads/flowers/flowers/dandelion"

PREPROCESS THE DATASET

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam


In [ ]:

# Set the image size
img_size = (224, 224)

# Set up data generators for training, validation, and testing
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)


In [ ]:

val_datagen = ImageDataGenerator(rescale=1./255)

test_datagen = ImageDataGenerator(rescale=1./255)

train_data_dir = os.path.join(dataset_dir, 'train')
val_data_dir = os.path.join(dataset_dir, 'val')
test_data_dir = os.path.join(dataset_dir, 'test')


In [ ]:
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=img_size,
    batch_size=32,
    class_mode='categorical'
)


In [ ]:

validation_generator = val_datagen.flow_from_directory(
    val_data_dir,
    target_size=img_size,
    batch_size=32,
    class_mode='categorical'
)

In [ ]:
test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=img_size,
    batch_size=32,
    class_mode='categorical'
)

BUILD THE CNN MODEL

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.applications import VGG16
from tensorflow.keras.optimizers import Adam


In [ ]:

# Load pre-trained VGG16 model, excluding the top (fully connected) layers
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))


In [ ]:

# Freeze the layers of VGG16
for layer in base_model.layers:
    layer.trainable = False

# Define the model
model = Sequential()

# Add the VGG16 base model
model.add(base_model)


In [ ]:

# Add custom layers
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))  # Dropout for regularization
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(5, activation='softmax'))  # Assuming 5 classes


In [ ]:

# Compile the model
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Summary of the model
model.summary()


TRAIN THE MODEL

In [ ]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=25,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size
)


EVALUATE THE MODEL

In [ ]:
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test accuracy: {test_acc}")


TO IMPROVE ACCURACY

In [ ]:
for layer in base_model.layers[-4:]:  # Unfreeze top 4 layers
    layer.trainable = True

# Recompile the model after unfreezing some layers
model.compile(optimizer=Adam(lr=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
from tensorflow.keras.layers import BatchNormalization

model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
